In [23]:
import pandas as pd
import numpy as np
# read all files
# importing packages
import pandas as pd
import glob
import missingno as msno
%matplotlib inline
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
# folder_path = 'x_all/'
# file_list = glob.glob(folder_path + "*.txt")
# f_name = [(re.search(r"\d+", i).group()) for i in file_list]
# main_dataframe = pd.DataFrame()

# for i in file_list:
#     data = pd.read_csv(i)
#     df = pd.DataFrame(data)
#     name = re.search(r"\d+", i).group()
#     df['Patient'] = int(name)
#     main_dataframe = pd.concat([main_dataframe,df])
# main_dataframe.to_csv("combined_data.csv", index = False)


In [24]:
# preprocessing data
# manipulate data structure
data = pd.read_csv("combined_data.csv")

# unmelt the data
data = data.set_index(['Patient', "Hour", "Variable"])["Value"].unstack().reset_index()
data = data.drop(columns = "ID")
data = data.rename(columns = {"Patient": "ID"})
data["Gender"] = data["Gender"].astype("bool")

# set subsets
train_out = pd.read_csv("train_outcome.csv")
train_out['Outcome'] = train_out['Outcome'].astype("bool")
train_ID = train_out.value_counts("ID").index.tolist()


# get submission test
sub = data[~data.ID.isin(train_ID)]

# get training set and test set
# split here by id
train,val = train_test_split(train_out,  stratify=train_out["Outcome"], 
                                                    test_size=0.2, random_state =10)

train = data.merge(train, how = "inner", on = "ID")
val = data.merge(val, how = "inner", on = "ID")

In [ ]:
# v = val.value_counts("ID").index.tolist()
# t = train.value_counts("ID").index.tolist()
# s = sub_df.value_counts("ID").index.tolist()

# val[val.ID.isin(t)]

In [3]:
# helper function for SOFA score

def get_PlateletsFactor(data):
    if data < 20:
        return 4
    elif data < 50:
        return 3
    elif data < 100:
        return 2
    elif data < 150:
        return 1
    else:
        return 0

def get_totalBF(data):
    if pd.isna(data):
        return 0
    if data < 1.2:
        return 0
    elif data < 2:
        return 1
    elif data < 6:
        return 2
    elif data < 12:
        return 3
    else:
        return 4
    
def get_renalFactor(data):
    if pd.isna(data):
        return 0
    
    if data < 1.2:
        return 0
    elif data < 2:
        return 1
    elif data < 3.5:
        return 2
    elif data < 5:
        return 3
    else:
        return 4
    
def get_MAP(data):
    if data < 70:
        return 1
    else:
        return 0
    
def calculate_SOFA(df):
    temp = df.copy()
    
    temp['PlateletsFactor'] = temp['Platelets'].apply(get_PlateletsFactor)
    temp['totalBF'] = temp['Bilirubin_total'].apply(get_totalBF)
    temp['renalFactor'] = temp['Creatinine'].apply(get_renalFactor)
    temp['BP'] = temp['MAP'].apply(get_MAP)
    temp["SOFA"] = temp['PlateletsFactor'] +   temp['totalBF'] +   temp['renalFactor'] + temp['BP']
    return temp["SOFA"]

In [ ]:
pd.DataFrame(train.isna().sum()/train.shape[0], columns = ['perc']).reset_index().sort_values('perc')

In [ ]:
train.corr().unstack().sort_values().reset_index().tail(60)

In [27]:


# # get SOFA Score
train['SOFA'] = calculate_SOFA(train)
val['SOFA'] = calculate_SOFA(val)

# function to pre-process data
def handle_missing_data(df):
    temp = df.copy()
    # drop cils with high corr
#     temp = temp.drop(columns =['Hgb','TroponinI','Fibrinogen' ,'Bilirubin_direct','BaseExcess'])
    
    m_h = temp.groupby(['ID'])['Hour'].max().reset_index().rename({"Hour":"Max_hour"}, axis = 1)
    temp = temp.merge(m_h, on = "ID", how = "inner")
    temp = temp.drop(columns = ['Hour'])
#     #drop hour 1 because a lot of missing data first hour
#     temp = temp[temp.Hour != 1]
    
    missing_val_perc = pd.DataFrame(train.isna().sum()/train.shape[0], columns = ['perc']).reset_index()
    col_dropped = missing_val_perc[missing_val_perc.perc > 0.90]['index']
    
    # fill na of age with same values
    cols_20 = ['HR', 'MAP', "O2Sat", 'Resp']
    
    cols_cat = ['Age', "Unit1", "Unit2"]
    
    # drop cols with a lot of missing values
#     temp = temp.drop(columns = col_dropped )
    
    # fill missing same value for each ID
    for i in cols_cat: 
        temp[i] = temp[i].fillna(temp.groupby('ID')[i].transform('mean'))

    
    # fcreate falg columns
#     for i in cols_20:
#         n_name = "flag_" + i
#         temp[n_name] = np.where(temp[i].isna(), 1,0)
# #         temp[i] = temp.groupby("ID")[i].apply(lambda x: x.interpolate())
 
            
    # forward fill the rest of cols
#     cols_ff = list(set(train.columns.tolist()) - set(cols_20)- set(cols_cat) - set(col_dropped))
    cols_ff = list(set(temp.columns.tolist()) - set(cols_20)- set(cols_cat))
    for i in temp.columns:
        if i != "Outcome":
            n_name = "flag_" + i
            temp[n_name] = np.where(temp[i].isna(), 1,0)
#         temp[i] = df.groupby(['ID'])[i].apply(lambda group: group.ffill())
 

    temp = temp.groupby('ID').apply(lambda x: x.ffill())
    temp = temp.groupby('ID').mean().reset_index()
#     temp = temp.fillna(temp.mean())
    
    return temp

train['Outcome'] = np.where(train['Outcome'] == True, 1, 0)
val['Outcome'] = np.where(val['Outcome'] == True, 1, 0)
train_df = handle_missing_data(train)
val_df = handle_missing_data(val)



# split the data to X, y
X_train = train_df.drop(columns = ['Outcome'])
y_train = train_df['Outcome']
X_test = val_df.drop(columns = ['Outcome'])
y_test = val_df['Outcome']


# # last step to impute leftover nan
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
# imputer = KNNImputer(n_neighbors=50)
# X_train = imputer.fit_transform(X_train)
# X_test = imputer.transform(X_test)

In [31]:
X_train

,ID,AST,Age,Alkalinephos,BUN,BaseExcess,Bilirubin_direct,Bilirubin_total,Calcium,Chloride,...,flag_SBP,flag_SaO2,flag_Temp,flag_TroponinI,flag_Unit1,flag_Unit2,flag_WBC,flag_pH,flag_SOFA,flag_Max_hour
0,2,NaN,66.67,NaN,12.166250,0.990000,NaN,NaN,NaN,107.468750,...,0.136364,0.909091,0.590909,1.000000,0.0,0.0,0.909091,0.909091,0.0,0.0
1,3,NaN,79.74,NaN,9.179091,2.123000,NaN,NaN,8.430196,109.641136,...,0.120690,0.879310,0.500000,1.000000,0.0,0.0,0.931034,0.810345,0.0,0.0
2,5,36.000000,60.74,58.000000,12.160155,NaN,NaN,1.300000,6.567364,NaN,...,0.082090,0.895522,0.768657,0.985075,1.0,1.0,0.955224,0.895522,0.0,0.0
3,7,19.296667,65.53,52.720000,17.274762,NaN,NaN,0.900000,9.100000,105.768571,...,0.058824,1.000000,0.764706,1.000000,1.0,1.0,0.941176,1.000000,0.0,0.0
4,13,40.607857,30.79,45.535714,14.971429,NaN,NaN,1.091429,7.258056,109.612857,...,0.162791,1.000000,0.744186,1.000000,0.0,0.0,0.930233,1.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12110,21625,NaN,45.40,NaN,13.778889,NaN,NaN,NaN,7.390000,NaN,...,0.150000,1.000000,0.050000,1.000000,0.0,0.0,0.900000,1.000000,0.0,0.0
12111,21628,NaN,61.44,NaN,14.000000,NaN,NaN,NaN,NaN,104.014074,...,0.607143,1.000000,0.750000,1.000000,0.0,0.0,0.964286,1.000000,0.0,0.0
12112,21629,NaN,63.82,NaN,23.535714,NaN,NaN,NaN,7.520370,NaN,...,0.196429,0.928571,0.517857,1.000000,0.0,0.0,0.964286,0.946429,0.0,0.0
12113,21630,NaN,65.41,NaN,42.466667,-4.222381,NaN,NaN,7.987879,104.863571,...,0.069767,0.976744,0.093023,1.000000,0.0,0.0,0.953488,0.860465,0.0,0.0


In [29]:
# some model
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc 
# cv for unbalac
cv = StratifiedKFold(n_splits=5)

#list of tuned params
n_estimators = np.arange(1, 201).tolist()
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = np.arange(2, 40, 5).tolist()
min_samples_leaf = np.arange(1, 40, 5).tolist()
max_iter = np.arange(50, 200, 11)



In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}


In [ ]:
# Random forest
rfc = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter =200, cv = cv, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
y_pred = rf_random.predict(X_test)
np.mean(y_pred!=y_test)

In [ ]:
# get auc
y_pred_prob = rf_random.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1])
roc_auc =  auc(fpr, tpr)
roc_auc

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
FPR = fp / (fp + tn)
FNR = fn / (fn + tp)
(FPR + FNR) * 0.5

In [36]:
import xgboost as xgb
# xgb classifier
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 2],
               'alpha': [0, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 2],
                  'gamma': [0, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 2],
              'scale_pos_weight': np.arange(1, 15),
              'subsample': np.linspace(0,1, num = 20)}
xgb_cl = xgb.XGBClassifier()
xgb_random = RandomizedSearchCV(estimator = xgb_cl, param_distributions = random_grid, n_iter = 200, cv = cv, verbose=0, random_state=42, n_jobs = -1)
xgb_random.fit(X_train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           impor...
                                                         15, 16, 17, 18, 19, 20,
                                                         21, 22, 23, 24, 25, 26,
                                                         27, 28, 29, 30, ...],
                                        'scale_pos_weight': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'subsample': array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
       0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
       0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ])},
                   random_state=42)

In [37]:
xgb_random.best_params_

{'subsample': 0.631578947368421,
 'scale_pos_weight': 9,
 'n_estimators': 145,
 'max_depth': 20,
 'learning_rate': 0.05,
 'gamma': 0.001,
 'alpha': 0.05}

In [38]:
# get auc and ber
# xgb_random = xgb.XGBClassifier(subsample=  0.6842105263157894,
#                              scale_pos_weight = 9,
#                              n_estimators = 150,
#                              max_depth = 10,
#                              learning_rate = 0.1,
#                              gamma = 0.1,
#                              alpha = 0.5)
# xgb_random.fit(X_train, y_train)
y_pred_prob = xgb_random.predict_proba(X_test)
y_pred = xgb_random.predict(X_test)
np.mean(y_pred != y_test)
print(xgb_random.score(X_test, y_test))

y_pred_prob = xgb_random.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1])
roc_auc =  auc(fpr, tpr)
print(roc_auc)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
FPR = fp / (fp + tn) 
FNR = fn / (fn + tp)
print((FPR + FNR) * 0.5)

0.9412347309343018
0.9392664622994278
0.1726764333474021


In [ ]:
# hist xgb
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 2],
               'alpha': [0, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 2],
                  'gamma': [0, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 2],
              'scale_pos_weight': np.arange(1, 15),
              'subsample': np.linspace(0,1, num = 20)}
xgbrf_cl = xgb.XGBRFClassifier()
xgbrf_random = RandomizedSearchCV(estimator = xgbrf_cl, param_distributions = random_grid, n_iter = 200, cv = cv, verbose=2, n_jobs = -1)
xgbrf_random.fit(X_train, y_train)

In [ ]:
y_pred = xgbrf_random.predict(X_test)
np.mean(y_pred != y_test)
print(xgbrf_random.score(X_test, y_test))

y_pred_prob = xgbrf_random.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1])
roc_auc =  auc(fpr, tpr)
print(roc_auc)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
FPR = fp / (fp + tn) 
FNR = fn / (fn + tp)
print((FPR + FNR) * 0.5)

In [ ]:

from sklearn.ensemble import HistGradientBoostingClassifier

random_grid = {'max_iter': max_iter,
               'max_depth': max_depth,
               'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 2],
               'l2_regularization': [0, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 2],
                  'max_leaf_nodes': np.arange(30,100, 11).tolist(),
              'min_samples_leaf':  min_samples_leaf}
hxgb_cl = HistGradientBoostingClassifier()
hxgb_random = RandomizedSearchCV(estimator = hxgb_cl, param_distributions = random_grid, n_iter = 300, cv = cv, verbose=2, random_state=10, n_jobs = -1)
hxgb_random.fit(X_train, y_train)

In [ ]:
y_pred = hxgb_random.predict(X_test)
np.mean(y_pred != y_test)
hxgb_random.score(X_test, y_test)

In [ ]:
y_pred_prob = hxgb_random.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1])
roc_auc =  auc(fpr, tpr)
roc_auc

In [ ]:
# get BER 
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

FPR = fp / (fp + tn) 
FNR = fn / (fn + tp)
(FPR + FNR) * 0.5

In [16]:
X_train

,ID,AST,Age,Alkalinephos,BUN,BaseExcess,Bilirubin_direct,Bilirubin_total,Calcium,Chloride,...,flag_Bilirubin_direct,flag_PaCO2,flag_BaseExcess,flag_WBC,flag_Glucose,flag_flag_Resp,flag_DBP,flag_BUN,flag_flag_O2Sat,flag_SBP
0,2,NaN,66.67,NaN,12.166250,0.990000,NaN,NaN,NaN,107.468750,...,1.0,0.954545,0.909091,0.909091,0.909091,0.0,0.090909,0.909091,0.0,0.136364
1,3,NaN,79.74,NaN,9.179091,2.123000,NaN,NaN,8.430196,109.641136,...,1.0,0.810345,0.810345,0.931034,0.810345,0.0,0.431034,0.948276,0.0,0.120690
2,5,36.000000,60.74,58.000000,12.160155,NaN,NaN,1.300000,6.567364,NaN,...,1.0,0.895522,1.000000,0.955224,0.813433,0.0,0.097015,0.955224,0.0,0.082090
3,7,19.296667,65.53,52.720000,17.274762,NaN,NaN,0.900000,9.100000,105.768571,...,1.0,1.000000,1.000000,0.941176,0.941176,0.0,1.000000,0.941176,0.0,0.058824
4,13,40.607857,30.79,45.535714,14.971429,NaN,NaN,1.091429,7.258056,109.612857,...,1.0,1.000000,1.000000,0.930233,0.883721,0.0,0.348837,0.883721,0.0,0.162791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12110,21625,NaN,45.40,NaN,13.778889,NaN,NaN,NaN,7.390000,NaN,...,1.0,1.000000,1.000000,0.900000,0.850000,0.0,0.100000,0.900000,0.0,0.150000
12111,21628,NaN,61.44,NaN,14.000000,NaN,NaN,NaN,NaN,104.014074,...,1.0,1.000000,1.000000,0.964286,0.964286,0.0,0.571429,0.964286,0.0,0.607143
12112,21629,NaN,63.82,NaN,23.535714,NaN,NaN,NaN,7.520370,NaN,...,1.0,0.928571,1.000000,0.964286,0.589286,0.0,0.142857,0.964286,0.0,0.196429
12113,21630,NaN,65.41,NaN,42.466667,-4.222381,NaN,NaN,7.987879,104.863571,...,1.0,0.883721,0.860465,0.953488,0.837209,0.0,0.069767,0.930233,0.0,0.069767


In [33]:
# prepare for submission
#combine training and val sets
train_sub = pd.concat([train, val])
train_sub['Outcome'] = np.where(train_sub['Outcome'] == True, 1, 0)
train_sub = handle_missing_data(train_sub)
X_train_sub = train_sub.drop(columns = "Outcome")
y_train_sub = train_sub["Outcome"]

# format the submission data
sub_df = sub.copy()
sub_df['SOFA'] = calculate_SOFA(sub_df)
X_test_sub = handle_missing_data(sub_df)


# impute leftover nan
# imputer_sub = KNNImputer(n_neighbors=50)
# X_train_sub = imputer_sub.fit_transform(X_train_sub)
# X_test_sub = imputer_sub.transform(X_test_sub)



In [34]:
X_test_sub

,ID,AST,Age,Alkalinephos,BUN,BaseExcess,Bilirubin_direct,Bilirubin_total,Calcium,Chloride,...,flag_SBP,flag_SaO2,flag_Temp,flag_TroponinI,flag_Unit1,flag_Unit2,flag_WBC,flag_pH,flag_SOFA,flag_Max_hour
0,1,NaN,50.73,NaN,15.000000,NaN,NaN,NaN,NaN,NaN,...,0.117647,1.000000,0.823529,1.000000,1.0,1.0,0.941176,1.000000,0.0,0.0
1,8,22.000000,73.55,NaN,17.000000,NaN,NaN,0.800000,8.500000,NaN,...,0.058824,1.000000,0.823529,1.000000,1.0,1.0,0.941176,1.000000,0.0,0.0
2,9,NaN,52.71,NaN,12.000000,NaN,NaN,NaN,8.100000,NaN,...,0.090909,1.000000,0.090909,1.000000,0.0,0.0,0.954545,1.000000,0.0,0.0
3,10,30.594091,66.65,46.655909,11.696364,NaN,NaN,1.358636,7.675000,NaN,...,0.133333,1.000000,0.666667,0.977778,0.0,0.0,0.933333,1.000000,0.0,0.0
4,11,33.000000,84.08,NaN,82.875750,NaN,NaN,NaN,8.541000,107.011750,...,0.047619,1.000000,0.833333,1.000000,1.0,1.0,0.928571,1.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6485,21617,NaN,44.84,NaN,8.000000,NaN,NaN,NaN,8.500000,NaN,...,0.055556,1.000000,0.888889,1.000000,1.0,1.0,0.944444,1.000000,0.0,0.0
6486,21623,1244.082444,61.04,270.014667,13.320222,0.577561,NaN,1.082703,6.672703,92.529556,...,0.173913,0.934783,0.717391,1.000000,0.0,0.0,0.826087,0.782609,0.0,0.0
6487,21627,NaN,44.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.205128,1.000000,0.717949,1.000000,0.0,0.0,1.000000,1.000000,0.0,0.0
6488,21631,NaN,61.00,NaN,11.333793,-0.385833,NaN,NaN,NaN,104.215862,...,0.205128,0.923077,0.435897,1.000000,1.0,1.0,0.974359,0.743590,0.0,0.0


In [39]:
xgb_sub = xgb.XGBClassifier(subsample=  0.631578947368421,
                             scale_pos_weight = 9,
                             n_estimators = 145,
                             max_depth = 20,
                             learning_rate = 0.05,
                             gamma = 0.001,
                             alpha = 0.05)
xgb_sub.fit(X_train_sub, y_train_sub)
y_pred_sub = xgb_sub.predict(X_test_sub)
y_pred_prob_sub = xgb_sub.predict_proba(X_test_sub)

# {'subsample': 0.5263157894736842,
#  'scale_pos_weight': 5,
#  'n_estimators': 155,
#  'max_depth': 30,
#  'learning_rate': 0.1,
#  'gamma': 0.001,
#  'alpha': 0.5}

In [40]:
sub_res = pd.read_csv("test_nolabel.csv")
sub_res['Outcome'] = y_pred_sub
sub_res['Score'] = y_pred_prob_sub[:,1]
sub_res[["Outcome"]].value_counts()

Outcome
0          5810
1           680
dtype: int64

In [ ]:
{'subsample': 0.631578947368421,
 'scale_pos_weight': 9,
 'n_estimators': 145,
 'max_depth': 20,
 'learning_rate': 0.05,
 'gamma': 0.001,
 'alpha': 0.05}

In [41]:
sub_res.to_csv("submission_pred.csv", index = False)

In [ ]:
train_sub[["Outcome"]].value_counts()